In [19]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

In [20]:
# Funcionarios
path_func =  './data/20240801_Planilhas_com_respostas_mdc_OCULTO.xlsx'
sheet_func = '20240801_Planilhas_com_resposta'

# Projetos alocados
path_proj = './data/Pessoas_alocacoes_projetos_OCULTOS_projeto.xlsx'
sheet_proj = 'Pessoas_alocacoes_projetos_OCUL'

# Mapeamento Competencia -> Categoria
sheet_map_competencia = 'Competencia->Categoria'

# Mapeamento do campo Tipo de Desligamento com o Y
#sheet_map_tipo_deslig = 'Tipo Desligamento > Y'

# Carrega os dados
df_funcionarios = pd.read_excel(path_func, sheet_name=sheet_func)
df_projetos = pd.read_excel(path_proj, sheet_name=sheet_proj)
df_map_competencia = pd.read_excel(path_proj, sheet_name=sheet_map_competencia)
#df_map_tipo_deslig = pd.read_excel(path_func, sheet_name=sheet_map_tipo_deslig)

output_path = './output/planilha_funcionarios_vs_projetos.xlsx'

In [21]:
df_projetos = df_projetos.drop_duplicates(subset=['projeto']).reset_index(drop=True)

In [22]:
df_perfil = df_funcionarios.groupby(['pessoa', 'categoria_mdc'])['grau_habilidade_mdc'].mean().reset_index()

In [23]:
# Mapeia competência -> categoria
mapa_competencia_categoria = dict(zip(df_map_competencia['competencia'], df_map_competencia['categoria_mais_proxima']))

In [24]:
# Lista das colunas de competência nos projetos
col_competencias = [col for col in df_projetos.columns if col.startswith('competencia')]


In [25]:
ranking = []

for _, projeto in df_projetos.iterrows():
    projeto_id = projeto['projeto']
    competencias = projeto[col_competencias].dropna().tolist()
    if len(competencias) == 0:
        continue
    categorias_necessarias = set(mapa_competencia_categoria[comp] for comp in competencias if comp in mapa_competencia_categoria)

    for funcionario_id, grupo in df_perfil.groupby('pessoa'):
        categorias_func = set(grupo['categoria_mdc'])

        # Cobertura: quantas categorias exigidas o funcionário cobre
        cobertura = len(categorias_necessarias & categorias_func)

        # Média dos graus nas categorias relevantes
        media_graus = grupo[grupo['categoria_mdc'].isin(categorias_necessarias)]['grau_habilidade_mdc'].mean()

        ranking.append({
            'projeto_id': projeto_id,
            'funcionario_id': funcionario_id,
            'cobertura': cobertura,
            'media_grau': media_graus,
            'score': cobertura * (media_graus if not pd.isna(media_graus) else 0)
        })

In [26]:
df_ranking = pd.DataFrame(ranking)  # Converte a lista de dicionários em DataFrame
df_sorted = df_ranking.sort_values(by=['funcionario_id', 'score'], ascending=[True, False])

# Seleciona os top_3 projetos para cada funcionário
df_top_proj_por_func = df_sorted.groupby('funcionario_id').head(3).reset_index(drop=True)

df_top_proj_por_func.to_excel(output_path, index=False)